In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
#Speech-error-Rate

def generate_network(choice):
    onsets = ['f', 'l', 'd', 'k', 'm']
    words = ['fog', 'dog', 'cat', 'mat', 'log']
    #This is to incorporate the two different neighborhoods.
    if(choice==0):
        words.append('hat')
        sem_nodes  = 57
        onsets.append('h')
    else:
        words.append('rat')
        sem_nodes  = 54
        onsets.append('r')
    

    network_nodes = {}
    excitation_values = {}
    nodes = list(range(sem_nodes)) + words + onsets + ['ae', 'o'] + ['t', 'g']
    for node in nodes:
        network_nodes[node] = []
        excitation_values[node] = [0]

    for i in range(10):
        if(i>=2 and i<=4):
            network_nodes[i].append('cat')
            network_nodes['cat'].append(i)
            network_nodes[i].append('dog')
            network_nodes['dog'].append(i)
            if(choice==1):
                network_nodes[i].append('rat')
                network_nodes['rat'].append(i)
        else:
            network_nodes[i].append('cat')
            network_nodes['cat'].append(i)
    
    if(choice==0):
        for i in range(10, 57):
            if(i>=10 and i<17):
                node_word = 'dog'
            elif(i>=17 and i<27):
                node_word = 'hat'
            elif(i>=27 and i<37):
                node_word = 'mat'
            elif(i>=37 and i<47):
                node_word = 'log'
            else:
                node_word = 'fog'
            network_nodes[i].append(node_word)
            network_nodes[node_word].append(i)
    else:
        for i in range(10, 54):
            if(i>=10 and i<17):
                node_word = 'dog'
            elif(i>=17 and i<24):
                node_word = 'rat'
            elif(i>=24 and i<34):
                node_word = 'mat'
            elif(i>=34 and i<44):
                node_word = 'log'
            else:
                node_word = 'fog'
            network_nodes[i].append(node_word)
            network_nodes[node_word].append(i)

    for word in words:
        if(word=='cat'):
            onset = 'k'
        else:
            onset= word[0]
        network_nodes[onset].append(word)
        network_nodes[word].append(onset)
        if(word[1]=='a'):
            vowel='ae'
        else:
            vowel='o'
        network_nodes[vowel].append(word)
        network_nodes[word].append(vowel)
        coda = word[2]
        network_nodes[coda].append(word)
        network_nodes[word].append(coda)
    return network_nodes, excitation_values


In [4]:
q = 0.5
p = 0.10
n = 8
runs = 1000 

values_list = []

std_dev1 = 0.01
std_dev2 = 0.16

net1, exc_values1 = generate_network(0)
net2, exc_values2 = generate_network(1)

output_words = []
for seed in range(runs):
    if(seed< (9*runs/10)):
        words = ['fog', 'dog', 'cat', 'mat', 'log', 'hat']
        network_nodes, excitation_values = net1, exc_values1
    else:
        words = ['fog', 'dog', 'cat', 'mat', 'log', 'rat']
        network_nodes, excitation_values = net2, exc_values2
#query for 'cat'
    for key in network_nodes:
        excitation_values[key] = [0]
    for i in range(10):
        excitation_values[i] = [10]
    random_gen = np.random.default_rng(seed)

    for t in range(1, n):
        for key in network_nodes:
            modifiers_ = 0
            for connections in network_nodes[key]:
                if(excitation_values[connections][-1]<0):
                    pass
                else:
                    modifiers_ += p*excitation_values[connections][-1]
            noise = random_gen.normal(scale = std_dev1)
            excitation_values[key].append((1-q)*excitation_values[key][-1] + modifiers_ + noise)
            excitation_values[key][-1] += random_gen.normal(scale = std_dev2*abs(excitation_values[key][-1]))

    
    maxi_ = excitation_values['fog'][-1]
    target_word = 'fog'
    for word in words:
        if(excitation_values[word][-1]> maxi_):
            maxi_ = excitation_values[word][-1]
            target_word = word
    
    for key in network_nodes:
        excitation_values[key] = [0]
    
    excitation_values[target_word] = [100]
    for t in range(1, n+1):
        for key in network_nodes:
            modifiers_ = 0
            for connections in network_nodes[key]:
                if(excitation_values[connections][-1]<0):
                    pass
                else:
                    modifiers_ += p*excitation_values[connections][-1]
            noise = random_gen.normal(scale = std_dev1)
            excitation_values[key].append((1-q)*excitation_values[key][-1] + modifiers_ + noise)
            excitation_values[key][-1] += random_gen.normal(scale = std_dev2*max(excitation_values[key][-2], 0))
    
    onset = words[0][0]
    for word in words:
        if(word=='cat'):
            if(excitation_values['k'][-1]>excitation_values[onset][-1]):
                onset = 'k'
        else:
            if(excitation_values[word[0]][-1]>excitation_values[onset][-1]):
                onset = word[0]
    if(excitation_values['ae'][-1]>excitation_values['o'][-1]):
        vowel =  'a'
    else:
        vowel = 'o'
    if(excitation_values['t'][-1]>excitation_values['g'][-1]):
        coda =  't'
    else:
        coda = 'g'
    if(onset=='k'):
        onset='c'
    output_words.append(onset+vowel+coda)

pd.Series(output_words).value_counts()

cat    942
dog     21
rat     17
dat     14
cot      3
cag      3
dtype: int64